# 종목 정보 덤프

In [22]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 0. 종목정보덤프
print(">> 0. 시가총액 5천억 이상 ticker 덤프")

# 0.0. 기존 데이터 제거
print("0.0. 기존 데이터 제거")
cursor.execute("TRUNCATE cap")
cursor.execute("TRUNCATE stocks_price")
db.commit()

# 0.1 시가총액 업데이트
print("0.1. 시가총액 업데이트")
capDf = stock.get_market_cap_by_ticker('20211031')
capDf.index.names = ['ticker']
capDf = capDf.rename(columns={'종가':'close', '시가총액':'cap'}).drop(['거래량', '거래대금', '상장주식수'], axis=1)
capDf = capDf[capDf['cap'] >= 500000000000]
db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
conn = db_connection.connect()
capDf.to_sql(name='cap', con=db_connection, if_exists='append', index=True, index_label="ticker")
conn.close()
#### > 구매한 티커는 별도로 관리

# 0.2. 티커 조회
print("0.2. 티커 조회")
cursor.execute("SELECT ticker FROM cap WHERE cap >= '500000000000'")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 0.3. 기간 설정
print("0.3. 기간 설정")
start = '20210601'
end = str(datetime.now().date()).replace('-', '')
print("기간: (%s - %s)" %(start, end))

db.close()

# 0.4. 새로운 데이터 업데이트
print("0.4. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
for ticker in tickers:
  try:
    df = stock.get_market_ohlcv_by_date(start, end, ticker)
    time.sleep(0.1)
    df['ticker'] = ticker
    df = df.rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})    
    df.index.names = ['date']
    df = df[['ticker', 'open', 'high', 'low', 'close', 'volume']]

    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, ticker)
    pass

# 0.5. 거래이상 종목제거
print("0.5. 거래이상종목제거")
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("DELETE FROM stocks_price WHERE ticker IN (SELECT DISTINCT(ticker) FROM stocks_price WHERE volume = 0)")
db.close()

>> 0. 시가총액 5천억 이상 ticker 덤프
0.0. 기존 데이터 제거
0.1. 시가총액 업데이트
The date you entered 20211031 seems to be a holiday. PYKRX changes the date parameter to 20211029 to query the requested information.
0.2. 티커 조회
0.3. 기간 설정
기간: (20210601 - 20211101)
0.4. 새로운 데이터 업데이트 (20210601 - 20211101)


# 종목 정보 최신화

In [21]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 1. 종목 최신화
print(">> 1. 시가총액 5천억 이상 ticker 최신화")

# 1.0. 티커 조회
print("1.0. 티커 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 1.1. 업데이트 기간 설정
print("1.1. 업데이트 기간 설정")
date = str(datetime.now().date()).replace('-', '')
print("기간: (%s)" %(date))

# 1.2. 기존 데이터 제거
print("1.2. 기존 데이터 제거 (%s)" %(date))
cursor.execute("DELETE FROM stocks_price WHERE date = " + date)
db.commit()
db.close()

# 1.3. 새로운 데이터 업데이트
print("1.3. 새로운 데이터 업데이트 (%s)" %(date))
for ticker in tickers:
  try:
    df = stock.get_market_ohlcv_by_date(str(datetime.now().date() - timedelta(days=1)).replace('-', ''), date, ticker)
    time.sleep(0.1)
    df['ticker'] = ticker
    df = df.rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})    
    df.index.names = ['date']
    df = df[['ticker', 'open', 'high', 'low', 'close', 'volume']]

    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, ticker)
    pass

>> 1. 시가총액 5천억 이상 ticker 최신화
1.0. 티커 조회
1.1. 업데이트 기간 설정
기간: (20211101)
1.2. 기존 데이터 제거 (20211101)
1.3. 새로운 데이터 업데이트 (20211101)


# 볼린저 밴드 최신화

In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 2. 볼린저밴드 최신화
print(">> 2. 볼린저 밴드 계산")

# 2.0. 티커 조회
print("2.0. 티커 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 2.1. 업데이트 기간 설정
print("2.1. 업데이트 기간 설정")
cursor.execute("SELECT DISTINCT date FROM boll ORDER BY date DESC limit 2")
startList = list(map(lambda x: x[0].strftime("%Y%m%d"), cursor.fetchall()))
start = startList[0]
# start = '20210101'
end = str(datetime.now().date()).replace('-', '')
if start == end:
  start = startList[1]
print("기간: (%s - %s)" %(start, end))  

# 2.2. 기존 데이터 제거
print("2.2. 기존 데이터 제거 (%s - %s)" %(start, end))
cursor.execute("DELETE FROM boll WHERE date >" + start + " AND date <= " + end)
db.commit()

# 2.3. 새로운 데이터 업데이트
print("2.3. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
day = 20
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
    # Set Dataframe
    priceDf['period'] = day
    priceDf['low'] = 0
    priceDf['medium'] = 0
    priceDf['high'] = 0
    priceDf['bandWidth'] = 0.0
    priceDf['position'] = 0.0
    # Set bandWidth, Bollinger
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
      avg = copyDf.mean()
      std = copyDf.std()
      
      lo = round(avg - std * 2)
      me = round(avg)
      hi = round(avg + std * 2)
      bw = (hi - lo) / me
      pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
      priceDf.at[idx, 'low'] = lo
      priceDf.at[idx, 'medium'] = me
      priceDf.at[idx, 'high'] = hi
      priceDf.at[idx, 'bandWidth'] = bw
      priceDf.at[idx, 'position'] = pos

    bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
    
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    bollDf.to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1)
    pass

# MFI 최신화

In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

### 3. MFI 최신화
print(">> 3. MFI 최신화")

# 3.0. 티커 조회
print("3.0. 티커 조회")
cursor.execute("SELECT ticker FROM stocks_info WHERE cap >= '500000000000'")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 3.1. 업데이트 기간 설정
print("3.1. 업데이트 기간 설정")
# cursor.execute("SELECT DISTINCT date FROM mfi ORDER BY date DESC limit 2")
# startList = list(map(lambda x: x[0].strftime("%Y%m%d"), cursor.fetchall()))
# start = startList[0]
start = '20210101'
end = str(datetime.now().date()).replace('-', '')
# if start == end:
  # start = startList[1]
print("기간: (%s - %s)" %(start, end))  

# 3.2. 기존 데이터 제거
print("3.2. 기존 데이터 제거 (%s - %s)" %(start, end))
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("DELETE FROM mfi WHERE date >" + start + " AND date <= " + end)
db.commit()
day = 10

# 3.3. 새로운 데이터 업데이트
print("3.3. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
    # Set Dataframe
    priceDf['period'] = day
    priceDf['tp'] = 0
    priceDf['mfi'] = 0
    priceDf['mfi_diff'] = 0
    # Set typical price
    for idx in range(0, len(priceDf)):
      row = priceDf.iloc[idx]
      priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
    # Set MFI
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      positiveRMF = 0
      negativeRMF = 0          
      for i in range(idx, idx + day):
        today = priceDf.iloc[i]
        yesterday = priceDf.iloc[i + 1]
        if today['tp'] > yesterday['tp']:
          positiveRMF += today['tp'] * today['volume']
        elif today['tp'] < yesterday['tp']:
          negativeRMF += today['tp'] * today['volume']
      MFI = round(positiveRMF / (positiveRMF + negativeRMF) * 100)
      priceDf.at[idx, 'mfi'] = MFI
      if idx > 0:
        priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
    mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1, ticker)
    pass
db.close()